# Архитектуры CNN

Мы рассмотрели базовые компоненты из которых состоят современные сверточные нейронные сети, а так же техники их обучения.

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_1.png"  width="700">

На этом занятии рассмортим какие именно модели можно построить на основе этих компонент. 




<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_2.png"  width="700">

Как упомяналось на первой лекции современный бум нейростевых технологий начался в 2012 году когда AlexNet большим отрывом от конкурентов победила в ImageNet

## AlexNet


создатели: Alex Krizhevsky, Ilya Sutskever, Geoffrey E. Hinton
**University of Toronto**


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_3.png"  width="850">


Особенности
- Большие фильтры на первых слоях
- Обучалась на двух видеокартах  GTX580 ( ... )
- Уменьшение размерности
- 2 Полносвязанных слоя в конце

https://neurohive.io/ru/vidy-nejrosetej/alexnet-svjortochnaja-nejronnaja-set-dlja-raspoznavanija-izobrazhenij/

https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf

ImageNet Classification with Deep ConvolutionalNeural Networks



В AlexNet есть все компоненты, которые мы рассматривали ранее. Её архитектура состоит из пяти свёрточных слоёв, между которыми располагаются pooling-слои и слои нормализации, а завершают нейросеть три полносвязных слоя.


На схеме архитектуры все выходные изображения делятся на два одинаковых участка — это связано с тем, что нейросеть обучалась на старых GPU GTX580, у которых было всего 3 ГБ видеопамяти. Для обработки использовались две видеокарты, чтобы параллельно выполнять операции над двумя частями изображения.
Изначально на вход подаётся фотография размером 227×227×3, и размер свёрточных фильтров первого слоя — 11×11. Всего применяется 96 фильтров с шагом 4.

Пространственные размеры изображения сначала довольно сильно сжимаются, затем постепенно увеличивается число фильтров. В результате ширина и высота этого уже не изображения, а набора активаций признаков, сильно уменьшаются, после чего оно поступает на два полносвязных своя, где количество весов уже довольно большое.


AlexNet не получится использовать для классификации cifar10, потому что если начать так агрессивно уменьшать изображение размером 32Х32 px, то в определенный момент в него просто не поместится следующий фильтр, который нужно применить, и изображение просто исчезнет.

Структура некоторых (особенно старых) сетей, заточенных под ImageNet, напрямую зависит от размера изображений: если соотношение сторон позволит фильтрам поместиться, проблем не возникнет,  если же оно меньше и в какой-то момент размер уменьшится до 2Х2 или 1Х1, то фильтр 3Х3 просто не сработает. В современных сетях есть слой, который позволяет решить эту проблему.



На тот момент такая архитектура показала прорывную точность (ошибка упала с 20% до 15.4%).




<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_4_1.png"  width="750">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_4_2.png"  width="850">


Также стоит упомянуть о нескольких важных параметрах нейросети, необычных на тот момент:


* AlexNet — практически первая архитектура, в которой применяется нелинейность ReLU. Ранее использовались сигмоидальные функции, которые работали медленнее;

* Используется собственная нормализация (не столь универсальная, как Batch-нормализацию) с отдельными слоями. Как выяснилось позднее, они не дают значительного улучшающего эффекта и поэтому не распространены в современных архитектурах;

* На этапе предварительной обработки используется очень большое дополнение данных (аугментация);

* Dropout 0.5 (то есть при регуляризации отсеивается половина нейронов);

* Размер пакета — 128;

* SGD Momentum 0.9 (как показывают сегодняшние эксперименты, это не плохой вариант, но чтобы обучение сходилось, им требовалась с помощью эвристики ниже периодически обучать Learning rate);

* Скорость обучения — 1e−2, снижается в 10 раз вручную, если точность в какой-то момент перестаёт расти;

* Встречаются следующие рекомендации — изначально обучить сеть <font color=red >N(?)</font>, а затем добиться дополнительной точности в 2%, то можно вернуться к SGD Momentum и аккуратно поработать с ним с маленьким шагом;

* Затухание весов L2 — 5e−4;

* В архитектуре используется ансамбль из 7 CNN — это позволило снизить процент ошибок с 18,2% до 15,4%. (Ансамбль моделей — это когда обучается несколько моделей, а результат считается по среднему значению. Здесь используются 7 сетей, результат усредняется. Таким образом достаточно сильно снижается ошибка).


## ZFnet


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_5.png"  width="850">


Тюнингованный AlexNet

Matthew D. Zeilerzeiler@cs.nyu.eduDept. of Computer Science, Courant Institute, Rob Fergusfergus@cs.nyu.eduDept. of Computer Science, **Courant Institute, New York Universit**


В 2013 году выиграла соревнования созданная учеными из Йорского университета нейросеть ZFnet, достигнув результата 11.7% — в ней AlexNet использовалась в качестве основы, но с изменёнными параметрами и слоями. 

Отличия от AlexNet небольшие: 
* Немного поменялись размеры фильтров (было 11, стало 7); 
* Увеличилось общее количество фильтров;

В целом и количество слоев, и общая структура сети, когда слои свертки и пулинга перемежаются друг с другом, а затем идут два полносвязных слоя, сохранились.


## VGGNet

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_6.png"  width="850">

Karen Simonyan and Andrew Zisserman

Visual Geometry Group - **Oxford**

https://www.robots.ox.ac.uk/~vgg/research/very_deep/


* Появление "стандартных" блоков внутри модели
* Свертки 3x3





В 2014-ом году в Оксфорде была разработана модель VGGNet. Сеть получилась более точной и более глубокой.

На слайде выше изображены сети AlexNet и две версии VGG-16 и VGG-19 с 16 и 19 слоями соответственно. На соревнованиях победила более глубокая VGG19, достигнув более чем в два раза лучшего результата по сравнению с AlexNet.

Особенности: все сверточные слои имеют фильтры с активным полем размера 3Х3, они объединены в блоки, состоящие из некоторого количества сверток с разным (постепенно увеличивающимся) количеством фильтров. Затем идут слои пулинга. Идея базового блока внутри сети будет достаточно широко использоваться дальше. 

В 2014 году VGGNet достигла более чем в два раза лучшего результата по сравнению с AlexNet. 

С маленькими фильтрами мы получим не так много параметров, но при этом сможем гораздо эффективнее обрабатывать их.

Попробуем посчитать размер выходных изображений и число параметров точно так же, как мы это делали для AlexNet. Если вычислить объём памяти, необходимый для обработки одного изображения, то он будет равен примерно 100 МБ (только для прямого прохода). Это означает, что видеокарта с 5 ГБ памяти сможет обрабатывать всего 50 фотографий за раз.


Несколько фактов об архитектуре VGGNet:

Нейросеть заняла 2 место в задаче классификации и 1 место в локализации на соревновании ImageNet (при локализации необходимо не только классифицировать объект, но и обвести его в ограничивающие рамки).
Процедура обучения такая же, как у AlexNet.
Слои нормализации отсутствуют.
В прикладных задачах обычно используются архитектуры VGG16 или VGG19 (VGG19 работает лучше, но расходует больше памяти).

Чтобы понять, в чем смысл этих усовершенствований, нужно оценить количество памяти для хранения весов и количество операций, которые требуется выполнить при прямом прохождении сигнала через сеть.

### Ресурсы

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_7.png"  width="850">

Пример оценки (на AlexNet)

Большее количество слоев требует больше памяти, весов и вычислительных мощностей.

### Размер рецептивного поля


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_8.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_9.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_10.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_11.png"  width="850">


* Основная часть памяти расходуется на большие свертки в начальных слоях, где пространственные размеры(ширина и высота) велики

* Больше всего весов в полносвязанных слоях

* Вычислительные ресурсы нужны в первую очередь для сверток

## GooleNet
В отличие от предшествующих моделей разработанна в коммерческой компании, с целью реального применения. 

Поэтому оснвной упор бул сделан на эффективности.


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_12.png"  width="850">

GoogleNet — ещё более глубокая архитектура с 22 слоями. Целью Google было разработать нейросеть с наибольшей вычислительной эффективностью. Для этого они придумали так называемый модуль Inception — вся архитектура состоит из множества таких модулей, следующих друг за другом.

Также в GoogleNet нет полносвязных слоёв, и она содержит всего 5 миллионов параметров — в 12 раз меньше, чем у AlexNet.

В составе GoogleNet есть небольшая подсеть — Stem Network. Она состоит из трёх свёрточных слоёв с двумя pooling-слоями и располагается в самом начале архитектуры.

На схеме нейросети можно увидеть небольшие промежуточные «отростки» — это вспомогательные классификационные выходы для введения дополнительного градиента на начальных слоях.

Идея основного модуля Inception заключается в том, что он сам по себе является небольшой локальной сетью. Вся его работа состоит в параллельном применении нескольких фильтров на исходное изображение. Данные фильтров объединяются, и создаётся выходной сигнал, который переходит на следующий слой.

Но здесь же кроется проблема. Рассмотрим пример: возьмём упрощённую версию модуля Inception и попробуем посчитать размеры выходных изображений после применения каждого фильтра. При исходной фотографии 28×28×256 получим следующее:

(залить отдельное изображение)


В результате вычислений мы обнаружили, что наш небольшой блок выполняет 854 миллиона операций! При этом pooling-слои не уменьшают размер объекта — это означает, что после каждого объединения фильтров он будет только расти, и вычисления станут очень дорогостоящими.

Чтобы этого избежать, введены так называемые «узкие места» — слои с фильтром 1×1, уменьшающие глубину изображения.


 ### Inception module

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_13.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_14.png"  width="850">

### 1x1 Convolution

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_15.png"  width="850">

Сверка 1х1 может рассматриваться как альтернатива полносвязанному слою.

Alternatively, interpret it as applying the same FC layer on each input pixel preserves spatial dimensions, reduces depth!
Projects depth to lower dimension (combination of feature maps)


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_16.png"  width="850">

Использование таких модулей и отсутствие полносвязных слоёв делают GoogleNet очень эффективной и достаточно точной сетью. Но на этом развитие свёрточных архитектур не остановилось.

### "Stem network"

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_17.png"  width="850">


 - в первых слоях быстро уменьшаются пространственные размеры. 

### Global Average Pooling

Полносвязанные слои замененны на GAP.

* Меньше весов
* Не зависимость от размера входа
* Регуляризация


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_18.png"  width="850">

<a href="https://arxiv.org/pdf/1312.4400.pdf">2014 Network In Network</a>

https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/

### Затухание градиента

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_19.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_20.png"  width="850">

## ResNet

В 2015 году ResNet произвела настоящую революцию глубины нейросетей. Она состояла из 152 слоёв и снизила процент ошибок до 3,57% в соревновании классификации ImageNet. Это сделало её почти в два раза эффективнее GoogleNet.

Что же происходит с нейросетью, когда мы увеличиваем число слоёв? Можно ли, взяв обычную архитектуру вроде VGG, просто складывать всё больше и больше слоёв друг на друга и достигать лучшей точности? 

Нет, нельзя. Скорее всего, более глубокая нейросеть покажет даже худшие результаты как при обучении, так и при тестировании. И переобучение здесь не при чём, поскольку тогда тренировочная ошибка была бы низкой.



<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_21.png"  width="850">



<a href="https://arxiv.org/pdf/1512.03385.pdf">2015 Deep Residual Learning for Image Recognition</a>

Resudial connection

* сумма а не конкатенация
* Batch normalization присутствует


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_22.png"  width="850">

https://arxiv.org/pdf/1512.03385.pdf


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_23.png"  width="850">

identity =  Тождественное отображение

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_24.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_25.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_26.png"  width="850">

https://pytorch.org/docs/stable/_modules/torchvision/models/resnet.html

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_27.png"  width="850">


<a href="https://arxiv.org/abs/1901.09321">2019 Fixup Initialization: Residual Learning Without Normalization</a>

https://towardsdatascience.com/understanding-fixup-initialization-6bf08d41b427

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_28.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_29.png"  width="850">

## Feature extraction
<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_30.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_31.png"  width="850">

## Обзор моделей

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_32.png"  width="850">

https://arxiv.org/pdf/1810.00736.pdf


## DensNet
<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_33.png"  width="850">

## ResNeXt

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_34.png"  width="850">



<a href="https://arxiv.org/abs/1611.05431">2016 Aggregated Residual Transformations for Deep Neural Networks</a>

### Groupped Convolution

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_35.png"  width="850">

https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_36.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_37.png"  width="850">

https://towardsdatascience.com/review-resnext-1st-runner-up-of-ilsvrc-2016-image-classification-15d7f17b42ac

## WideResNet


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_38.png"  width="850">



<a href="https://arxiv.org/abs/1605.07146">2016 Wide Residual Networks</a>

dropout внутри блоков

## SENet

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_39.png"  width="850">



<a href="https://arxiv.org/abs/1709.01507">2017 Squeeze-and-Excitation Networks</a>

https://towardsdatascience.com/squeeze-and-excitation-networks-9ef5e71eacd7

MobileNet
<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_40.png"  width="850">

### Depthwise separable convolution

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_41.png"  width="850">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_42.png"  width="850">

### Shuffled Grouped Convolution

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_43.png"  width="850">

## Neural Architecture Search

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_44.png"  width="850">

## Visual Transformers

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_45.png"  width="850">




<a href="https://arxiv.org/abs/2006.03677">2020 Visual Transformers: Token-based Image Representation and Processing for Computer Vision</a>


old version: https://syncedreview.com/2020/06/12/facebook-and-uc-berkeley-boost-cv-performance-and-lower-compute-cost-with-visual-transformers/

MAC (multiply and accumulate operations) 


<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_46.png"  width="850">